In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt

In this problem, we solve the following ridge regression problem:
$$\min_{x \in \mathbb{R}^{n}}f_{\lambda}(x) \qquad \text{where} \qquad f_{\lambda}(x) \doteq \frac{1}{2}\left\{\frac{1}{m}\|Ax - y\|_{2}^{2} + \lambda \|x\|_{2}^{2}\right\}$$
via gradient descent with step size $\eta > 0$.

# Computation Functions

In [ ]:
def make_f(A, y, lmbda):
    m, n = A.shape
    return lambda x: 0.5 * ((np.linalg.norm(A @ x - y) ** 2) / m + lmbda * np.linalg.norm(x) ** 2)

In [ ]:
def ridge_regression_sol(A, y, lmbda):
    m, n = A.shape
    # Closed-form ridge regression solution to compare to
    return np.linalg.inv(A.T @ A + lmbda * m * np.eye(A.shape[1])) @ A.T @ y

In [ ]:
def sigma_max(A):
    S = np.linalg.svd(A, full_matrices=True, compute_uv=False)
    return S[0]

def sigma_min(A):
    S = np.linalg.svd(A, full_matrices=True, compute_uv=False)
    if S.shape[0] < A.shape[1]:
        return np.float64(0.0)
    else:
        return S[-1]

In [ ]:
def gd(x0, grad_fn, eta, T):
    x_history = []
    x = x0
    for t in range(T):
        x_history.append(copy.deepcopy(x))
        x = ...  # TODO: update rule of GD
    return np.array(x_history)

In [ ]:
def eta_opt(A, lmbda):
    m, n = A.shape
    return 2 * m / (sigma_max(A) ** 2 + sigma_min(A) ** 2 + 2 * lmbda * m)

In [ ]:
def ridge_regression_gd(A, y, lmbda, eta, T):
    m, n = A.shape
    def grad_fn(x):
        return ...  # TODO: gradient of our ridge objective
    x0 = np.random.standard_normal((n,))
    return gd(x0, grad_fn, eta, T)

In [ ]:
def ridge_regression_sgd(A, y, lmbda, eta, T):
    m, n = A.shape
    def grad_fn(x):
        return ... # TODO: SGD gradient estimate for our ridge objective 
        # Hint: to generate a random index, you might want to check out np.random.randint
    x0 = np.random.standard_normal((n,))
    return gd(x0, grad_fn, eta, T)

In [ ]:
def time_average(x_history):
    time_sum = np.cumsum(x_history, axis=0)
    time_avg = time_sum / (1 + np.arange(x_history.shape[0]).reshape(-1, 1))
    return time_avg

# Plotting Functions

In [ ]:
def plot_data_1d(A, y):
    plt.xlabel("$a$")
    plt.ylabel("$b$")
    plt.plot(A[:, 0], y, marker="o", linestyle="none")

In [ ]:
def plot_data_2d(A, y):
    ax = plt.figure().add_subplot(projection='3d')
    ax.set_xlabel("$a_{1}$")
    ax.set_ylabel("$a_{2}$")
    ax.plot(A[:, 0], A[:, 1], zs=y, marker="o", linestyle="none")

In [ ]:
def plot_loss_curves_1d(A, y, lmbdas, x_star):
    m, n = A.shape
    assert n == 1
    for lmbda in lmbdas:
        f = make_f(A, y, lmbda)
        x_range = np.linspace(start=x_star - 10, stop=x_star + 10, num=1000).reshape(-1, 1)
        fx_range = np.array([f(x) for x in x_range])

        x_RR = ridge_regression_sol(A, y, lmbda)
        fx_RR = f(x_RR)

        plt.xlabel("$x$")
        plt.ylabel("$f_{\lambda}(x)$")
        plt.plot(x_range, fx_range, label = "$\lambda = %s$"%lmbda)

        plt.plot(x_RR, fx_RR, 'o')
    plt.legend()

In [ ]:
def plot_history_lines_1d(A, y, x_history, x_star, ts, time_avg=False):
    plt.xlabel("$a$")
    plt.ylabel("$b$")
    plt.plot(A[:, 0], y, ".")
    
    if time_avg:
        x_history = time_average(x_history)

    for t in ts:
        x_t = x_history[t-1][0]
        plt.plot([-1, 1], [-x_t, x_t], label="$x_{%s}$"%t)
    plt.plot([-1, 1], [-x_star, x_star], label="$x_{\lambda}^{\star}$")
    plt.legend()

In [ ]:
def plot_x_trajectory_1d(f, x_history, x_star, time_avg=False):
    if time_avg:
        x_history = time_average(x_history)
        
    x_range = np.linspace(start=x_star - 3, stop=x_star + 3, num=1000).reshape(-1, 1)
    fx_range = np.array([f(x) for x in x_range])

    plt.xlabel("$x$")
    plt.ylabel("$f_{\lambda}(x)$")
    plt.plot(x_range, fx_range)
    
    plt.plot(x_star, f(x_star), 'o')

    f_history = np.array([f(x) for x in x_history]).reshape(-1, 1)

    plt.quiver(x_history[:-1], f_history[:-1],\
           x_history[1:]-x_history[:-1], f_history[1:]-f_history[:-1],\
           scale_units='xy', angles='xy', scale=1)

In [ ]:
def plot_x_trajectory_2d(f, x_history, x_star, time_avg=False):
    if time_avg:
        x_history = time_average(x_history)
        
    plt.xlabel("$x_{1}$")
    plt.ylabel("$x_{2}$")
    
    plt.plot(x_star[0], x_star[1], 'o')
    
    plt.quiver(x_history[:-1,0], x_history[:-1,1],\
           x_history[1:,0]-x_history[:-1,0], x_history[1:,1]-x_history[:-1,1],\
           scale_units='xy', angles='xy', scale=1)
    
    xmin, xmax, ymin, ymax = plt.axis()
    x_range = np.linspace(start=xmin, stop=xmax, num=100)
    y_range = np.linspace(start=ymin, stop=ymax, num=100)
    xv, yv = np.meshgrid(x_range, y_range)
    z = np.stack((xv, yv), axis=-1)
    fz = np.zeros((z.shape[0], z.shape[1]))
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            fz[i][j] = f(z[i][j])
    min_fz = np.min(fz)
    max_fz = np.max(fz)
    level1 = min_fz + 0.05 * (max_fz - min_fz)
    level2 = min_fz + 0.15 * (max_fz - min_fz)
    plt.contour(x_range, y_range, fz, levels=[level1, level2])

In [ ]:
def plot_convergence_x(A, x_history, x_star, time_avg=False):
    if time_avg:
        x_history = time_average(x_history)
        
    x_history = x_history.T  # now it's a (N x T) matrix where each column is an x_t
    
    plt.xlabel("$t$")
    plt.ylabel("$|x_{t} - x_{\lambda}^{\star}|_{i}$")
    plt.yscale("log")
    for i in range(A.shape[1]):
        dist_xi = np.abs(x_history[i] - x_star[i])
        plt.plot(dist_xi, label="i=%s" % (i + 1))
    plt.legend()

In [ ]:
def plot_convergence_z(A, x_history, x_star, time_avg=False):
    if time_avg:
        x_history = time_average(x_history)
        
    U, S, Vt = np.linalg.svd(A)
    x_history = x_history.T  # now it's a (N x T) matrix where each column is an x_t
    z_history = Vt @ x_history  # project each x_t onto the V basis
    z_star = Vt @ x_star
    
    plt.xlabel("$t$")
    plt.ylabel("$|v_{i}^{\\top}(x_{t} - x_{\lambda}^{\star})|$")
    plt.yscale("log")
    for i in range(A.shape[1]):
        dist_zi = np.abs(z_history[i] - z_star[i])
        plt.plot(dist_zi, label="i=%s, $\sigma_{%s} = %s$" % (i + 1, i + 1, round(S[i], 2)))
    plt.legend()

In [ ]:
def plot_convergence_f(f, x_history, x_star, time_avg=False):
    if time_avg:
        x_history = time_average(x_history)
    
    f_history = np.array([f(x_history[i]) for i in range(len(x_history))])
    f_star = f(x_star)
    
    plt.xlabel("$t$")
    plt.ylabel("$|f_{\lambda}(x_{t}) - f_{\lambda}(x_{\lambda}^{\star})|$")
    dist_f = np.abs(f_history - f_star)
    plt.yscale("log")
    plt.plot(dist_f)

In [ ]:
def convergence_heatmap(A, y, etas, lmbdas, gd_fn, T, convergence_tol, time_avg):
    eta_lmbda_matrix = np.zeros(shape=(len(etas), len(lmbdas)))
    for i in range(len(etas)):
        for j in range(len(lmbdas)):
            x_history = gd_fn(A, y, lmbdas[j], etas[i], T)
            if time_avg:
                x_history = time_average(x_history)
            x_T = x_history[-1]
            x_rr = ridge_regression_sol(A, y, lmbdas[j])
            eta_lmbda_matrix[i, j] = 1.0 if np.linalg.norm(x_rr - x_T) < convergence_tol else 0.0

    plt.xlabel("$\eta$")
    plt.ylabel("$\lambda$")

    plt.xticks(range(len(etas)), etas, rotation = 45)
    plt.yticks(range(len(lmbdas)), lmbdas)

    plt.imshow(eta_lmbda_matrix)
    cbar = plt.colorbar()
    cbar.set_ticks([0,1])
    cbar.set_ticklabels(["Does not \n converge", "Converges"])
    
def convergence_heatmap_gd(A, y, etas, lmbdas, T=200, convergence_tol=0.1, time_avg=False):
    convergence_heatmap(A,  y, etas, lmbdas, ridge_regression_gd, T, convergence_tol, time_avg)
    
def convergence_heatmap_sgd(A, y, etas, lmbdas, T=200, convergence_tol=0.1, time_avg=False):
    convergence_heatmap(A,  y, etas, lmbdas, ridge_regression_sgd, T, convergence_tol, time_avg)

# Part 1: $n = 1$

In [ ]:
m_1 = 50
n_1 = 1
noise_std_1 = 0.5
x_star_1 = np.array([3]).reshape(n_1,)  # (n,)
A_1 = np.random.uniform(low=-1, high=1, size=(m_1, n_1))  # (m, n)
y_1 = A_1 @ x_star_1 + noise_std_1 * np.random.standard_normal((m_1, ))  # (m,)

In [ ]:
# Just to show us what the data looks like
plot_data_1d(A_1, y_1)

In [ ]:
plot_loss_curves_1d(A_1, y_1, [0, 0.1, 0.5], x_star_1)

In [ ]:
lmbda_1 = 0.1  # TODO: Change this and write your thoughts in the answer PDF
eta_1 = 1.1 * eta_opt(A_1, lmbda_1)  # TODO: Change this and write your thoughts in the answer PDF

In [ ]:
T = 200
f_1 = make_f(A_1, y_1, lmbda_1)
x_history_1 = ridge_regression_gd(A_1, y_1, lmbda_1, eta_1, T)  # TODO: Replace with SGD and write down your thoughts.
x_rr_1 = ridge_regression_sol(A_1, y_1, lmbda_1)

Here we visualize the regression lines obtained from our gradient descent iterates compared to the regression line obtained from the closed-form solution.

In [ ]:
plot_history_lines_1d(A_1, y_1, x_history_1, x_rr_1, [1, 5, 10, 20])

Here we visualize the convergence of $x_{t}$ within the standard basis.

In [ ]:
plot_convergence_x(A_1, x_history_1, x_rr_1)

Here we visualize the convergence of $x_{t}$ within the $V$ basis. Since we are in 1D, the two plots look the same. But they will soon diverge as we move to higher dimensions.

In [ ]:
plot_convergence_z(A_1, x_history_1, x_rr_1)

Here we visualize the convergence of $f_{\lambda}(x_{t})$.

In [ ]:
plot_convergence_f(f_1, x_history_1, x_rr_1)

The next plot shows how $x_{t}$ bounces around the loss landscape, more precisely plotting $(x_{t}, f_{\lambda}(x_{t}))$. Arrows point from $(x_{t}, f_{\lambda}(x_{t}))$ to $(x_{t + 1}, f_{\lambda}(x_{t + 1}))$ for all $t$. The optimal point $(x_{\lambda}^{\star}, f_{\lambda}(x_{\lambda}^{\star})$ is highlighted.

In [ ]:
plot_x_trajectory_1d(f_1, x_history_1, x_rr_1)

We now plot the convergence of gradient descent for different values of $\eta$ and $\lambda$.

In [ ]:
etas = [1e-3, 2*1e-3, 5*1e-3, 1e-2, 2*1e-2, 5*1e-2, 1e-1, 2*1e-1, 5*1e-1, 1] 
lmbdas = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]

convergence_heatmap_gd(A_1, y_1, etas, lmbdas, T = 100, convergence_tol=1)

# Part 2: $n = 2$

In [ ]:
m_2 = 50
n_2 = 2
noise_std_2 = 0.5
x_star_2 = np.array([3, 1]).reshape(n_2,)  # (n,)
A_2 = np.random.uniform(low=-1, high=1, size=(m_2, n_2))  # (m, n)
y_2 = A_2 @ x_star_2 + noise_std_2 * np.random.standard_normal((m_2, ))  # (m,)

In [ ]:
# Just to show us what the data looks like
plot_data_2d(A_2, y_2)

In [ ]:
lmbda_2 = 0.0  # TODO: Change this and write your thoughts in the answer PDF
eta_2 = 1.5 * eta_opt(A_2, lmbda_2)  # TODO: Change this and write your thoughts in the answer PDF

In [ ]:
T = 200
f_2 = make_f(A_2, y_2, lmbda_2)
x_history_2 = ridge_regression_gd(A_2, y_2, lmbda_2, eta_2, T)  # TODO: Replace with SGD and write down your thoughts.
x_rr_2 = ridge_regression_sol(A_2, y_2, lmbda_2)

Here we visualize the convergence of $x_{t}$ within the standard basis.

In [ ]:
plot_convergence_x(A_2, x_history_2, x_rr_2)

Here we visualize the convergence of $x_{t}$ within the $V$ basis. Note that while distance curves in the above figure might be non-monotonic or irregular, the below figure curves will be monotonic decreasing. (Why? Look at the equations in the PDF.)

In [ ]:
plot_convergence_z(A_2, x_history_2, x_rr_2)

Here we plot the convergence of $f_{\lambda}(x_{t})$.

In [ ]:
plot_convergence_f(f_2, x_history_2, x_rr_2)

We now show how $x_{t}$ bounces around $\mathbb{R}^{2}$. Arrows point from $x_{t}$ to $x_{t + 1}$. The optimal point $(x_{\lambda}^{\star}, f_{\lambda}(x_{\lambda}^{\star}))$ is highlighted, as well as two level sets of the objective function.

In [ ]:
plot_x_trajectory_2d(f_2, x_history_2, x_rr_2)

We now plot the convergence of gradient descent for different values of $\eta$ and $\lambda$.

In [ ]:
etas = [1e-3, 2*1e-3, 5*1e-3, 1e-2, 2*1e-2, 5*1e-2, 1e-1, 2*1e-1, 5*1e-1, 1] 
lmbdas = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]

convergence_heatmap_gd(A_2, y_2, etas, lmbdas, T = 100, convergence_tol=1)

# Part 3: $n \gg 2$

Here, we cannot use our visual intuition to see the trajectory of $x_{t}$. But we can use the principles we derived to observe convergence anyways.

In [ ]:
m_10 = 50
n_10 = 10
noise_std_10 = 0.5
x_star_10 = np.random.uniform(low=1, high=3, size=(n_10,))  # (n,)
A_10 = np.random.uniform(low=-1, high=1, size=(m_10, n_10))  # (m, n)
y_10 = A_10 @ x_star_10 + noise_std_10 * np.random.standard_normal((m_10, ))  # (m,)

In [ ]:
lmbda_10 = 0.01  # TODO: Change this and write your thoughts in the answer PDF
eta_10 = 1.1 * eta_opt(A_10, lmbda_10)  # TODO: Change this and write your thoughts in the answer PDF

In [ ]:
T = 200
f_10 = make_f(A_10, y_10, lmbda_10)
x_history_10 = ridge_regression_gd(A_10, y_10, lmbda_10, eta_10, T)  # TODO: Replace with SGD and write down your thoughts.
x_rr_10 = ridge_regression_sol(A_10, y_10, lmbda_10)

Here we visualize the convergence of $x_{t}$ within the standard basis.

In [ ]:
plot_convergence_x(A_10, x_history_10, x_rr_10)

Here we visualize the convergence of $x_{t}$ within the $V$ basis. Note that the below figure curves will be monotonic decreasing.

In [ ]:
plot_convergence_z(A_10, x_history_10, x_rr_10)

Here we plot the convergence of $f_{\lambda}(x_{t})$.

In [ ]:
plot_convergence_f(f_10, x_history_10, x_rr_10)

We now plot the convergence of gradient descent for different values of $\eta$ and $\lambda$.

In [ ]:
etas = [1e-3, 2*1e-3, 5*1e-3, 1e-2, 2*1e-2, 5*1e-2, 1e-1, 2*1e-1, 5*1e-1, 1] 
lmbdas = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]

convergence_heatmap_gd(A_10, y_10, etas, lmbdas, T = 5000, convergence_tol=0.01)